In [1]:
import os
os.chdir(".\\Case Studies\\Human Activity Recognition")

In [2]:
import pandas as pd
import numpy as np

feature_names = ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x', 'body_gyro_y', 
                 'body_gyro_z', 'total_acc_x', 'total_acc_y', 'total_acc_z']

In [3]:
def get_x_data(type_data = 'train'):
    
    x_data = []
    
    for feature in feature_names:
        file_name = f'.\\UCI HAR Dataset\\UCI HAR Dataset\\{type_data}\\Inertial Signals\\{feature}_{type_data}.txt'
        data = pd.read_csv(file_name, delim_whitespace = True, header = None).as_matrix()
        x_data.append(data)
    
    return np.transpose(x_data, (1, 2, 0))

def get_y_data(type_data = 'train'):
    
    y_data = []
    
    file_name = f'.\\UCI HAR Dataset\\UCI HAR Dataset\\{type_data}\\y_{type_data}.txt'
    return pd.get_dummies(pd.read_csv(file_name, delim_whitespace = True, header = None)[0]).as_matrix()

In [4]:
x_train = get_x_data('train')
x_test = get_x_data('test')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [14]:
y_train = get_y_data('train')
y_test = get_y_data('test')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [6]:
vector_size = len(x_train[0])
feature_count = len(x_train[0][0])

In [42]:
from sklearn.metrics import confusion_matrix

column_list = ['Dropout', 'Multiple LSTM Layers', 'Accuracy']
results = pd.DataFrame(columns = column_list)

def print_results(model, x_test, y_test, dropout, multiple_layers):
    global results
    y_pred = model.predict(x_test)
    y_pred_modified = [prediction.argmax() for prediction in y_pred]
    y_test_modified = [actual_value.argmax() for actual_value in y_test]

    print('---------------------------------------------------------------------------------------------')
    print('Results for model with dropout: {}, multiple_layers: {}'.format(dropout, multiple_layers))
    print(confusion_matrix(y_test_modified, y_pred_modified))
    result = model.evaluate(x_test, y_test)
    print('Accuracy: {}'.format(result[1]))
    print('---------------------------------------------------------------------------------------------')
    result = {'Dropout': dropout, 'Multiple LSTM Layers': multiple_layers, 'Accuracy': result[1]}
    results = results.append(result, ignore_index = True)

In [59]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import LSTM

lstm_count_single = 32
lstm_count_multiple = 15
epochs = 30
batch_size = 16

def fit_model(dropout, multiple_layers, x_train, y_train, x_test, y_test):
    print('Running for dropout: {}, layers: {}'.format(dropout, multiple_layers))
    if multiple_layers:
        lstm_count = lstm_count_multiple
    else:
        lstm_count = lstm_count_single
    model = Sequential()
    model.add(LSTM(lstm_count, return_sequences=True, input_shape = (vector_size, feature_count)))
    model.add(Dropout(dropout))
    if multiple_layers:
        model.add(LSTM(lstm_count, return_sequences=True))
        model.add(Dropout(dropout))
    model.add(Flatten())    
    model.add(Dense(6, activation='softmax'))
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=epochs)
    print_results(model, x_test, y_test, dropout, multiple_layers)

In [63]:
dropouts = [0.25, 0.40, 0.50]
multiple_layers = [False, True]

combinations = [(dropout, layers) for dropout in dropouts for layers in multiple_layers]

for dropout, layers in combinations:
    fit_model(dropout, layers, x_train, y_train, x_test, y_test)

Running for dropout: 0.25, layers: False
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.5625 - acc: 0.7758 - val_loss: 0.4389 - val_acc: 0.8660
Epoch 2/30
7352/7352 [==============================] - 56s 8ms/step - loss: 0.1681 - acc: 0.9319 - val_loss: 0.3786 - val_acc: 0.8992
Epoch 3/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.1443 - acc: 0.9402 - val_loss: 0.3687 - val_acc: 0.8979
Epoch 4/30
7352/7352 [==============================] - 49s 7ms/step - loss: 0.1260 - acc: 0.9497 - val_loss: 0.3034 - val_acc: 0.9131
Epoch 5/30
7352/7352 [==============================] - 49s 7ms/step - loss: 0.1294 - acc: 0.9468 - val_loss: 0.3724 - val_acc: 0.8962
Epoch 6/30
7352/7352 [==============================] - 49s 7ms/step - loss: 0.1191 - acc: 0.9506 - val_loss: 0.3544 - val_acc: 0.8992
Epoch 7/30
7352/7352 [==============================] - 49s 7ms/step - loss: 0.1166 - acc: 0.9502 - v

7352/7352 [==============================] - 94s 13ms/step - loss: 0.1103 - acc: 0.9498 - val_loss: 0.3660 - val_acc: 0.9175
Epoch 18/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1177 - acc: 0.9457 - val_loss: 0.6127 - val_acc: 0.8534
Epoch 19/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1215 - acc: 0.9471 - val_loss: 0.3251 - val_acc: 0.9050
Epoch 20/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1268 - acc: 0.9494 - val_loss: 0.2771 - val_acc: 0.9070
Epoch 21/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1105 - acc: 0.9494 - val_loss: 0.4971 - val_acc: 0.8945
Epoch 22/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1095 - acc: 0.9529 - val_loss: 0.3134 - val_acc: 0.9118
Epoch 23/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1076 - acc: 0.9517 - val_loss: 0.3199 - val_acc: 0.9094
Epoch 24/30
7352/7352 [==============================

In [64]:
results

,Dropout,Multiple LSTM Layers,Accuracy
0,0.25,False,0.926026
1,0.25,False,0.911096
2,0.25,True,0.891754
3,0.40,False,0.882592
4,0.40,True,0.917543
5,0.50,False,0.906685
6,0.50,True,0.916186
